# Chapter 7 이미지 인식의 은총알 CNN

CNN`Convolution Neural Network`은 이미지, 음성 인식, 자연어 처리 등에서 강력한 성능을 발휘하고 있다. 이번 장에서는 CNN을 이용해서 MNIST 데이터를 학습해보고 기본적인 신경망보다 성능이 얼마나 좋아지는지 확인해보자.

## 7.1 CNN 개념

![cnn-structure](images/cnn-structure.png)

CNN은 위 그림과 같이 컨볼루션 레이어`Convolution Layer`와 풀링 레이어`Pooling Layer`으로 이루어진다. 이 레이어들을 얼마나 어떻게 쌓느냐에 따라 성능이나 풀 수 있는 문제가 달라진다.

만약 데이터가 2D 이미지라면,

- 컨볼루션 레이어: 웨이트와 바이어스를 적용
- 풀링 레이어: 값들 중 하나를 선택

의 과정을 통해 지정한 영역의 데이터를 하나의 값으로 압축한다. 그림으로 보면 다음과 같다.

![c-p-layers](images/c-p-layers.png)

위 그림 중 컨볼루션 레이어의 빨간색 사각형을 윈도우라고 하고, 윈도우가 오른쪽/아래로 움직이는 단위를 스트라이드`Stride`라고 한다.

윈도우를 히든 레이어의 뉴런 하나로 압축하기 위해서, 컨볼루션 레이어는 윈도우 크기 만큼의 웨이트와 한개의 바이어스가 필요하다. 이 웨이트/바이어스 셋을 커널`Kernal` 혹은 필터`Filter`라고 한다. 이 커널은 모든 윈도우에 공통적으로 적용된다. 이 커널 개념 때문에 찾아야 하는 웨이트의 수가 적어져서(전체 사이즈가 아니라 윈도우 사이즈) 계산량이 빨라지게 된다. 물론 복잡한 이미지를 분석하기 위해서는 여러개의 커널을 사용해야 할 수도 있으며, 커널의 개수 역시 하이퍼파라메터이다.

## 7.2 모델 구현하기

### 데이터 입력

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

/Users/qodot/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### 입력값 모델

In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])  # None: 데이터 입력 개수, 28x28: 이미지 입력, 1: 특징 개수 (이미지가 흑백이라 색이 하나 뿐)
Y = tf.placeholder(tf.float32, [None, 10])  # 출력 값의 개수
keep_prob = tf.placeholder(tf.float32)  # dropout

### 첫 번째 CNN 레이어

첫 번째 CNN 레이어를 다음과 같은 모습으로 만들어본다.

![first-cnn](images/first-cnn.png)

#### 컨볼루션 레이어

In [3]:
# 3x3: 윈도우 크기
# 1: 입력의 커널 사이즈 (원본 이미지 한 장에 대해 컨볼루션 레이어를 적용하므로 1)
# 32: 출력의 커널 사이즈
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# padding='SAME': 이미지의 바깥으로 한 칸 더 움직이겠다는 뜻 (이미지의 외곽 특징도 더 잘 피악할 수 있다)
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

# 고수준 API를 이용하면 더 간단하게도 가능하다
# L1 = tf.layers.conv2d(X, 32, [3, 3])

#### 풀링 레이어

In [4]:
# 2x2: 윈도우 크기
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 고수준 API를 이용하면 더 간단하게도 가능하다
# L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])

### 두 번째 CNN 레이어

첫 번째 CNN 레이어 바로 뒤에 이어 붙일 두 번째 CNN 레이어를 다음과 같은 모습으로 만들어본다.

![second-cnn](images/second-cnn.png)

#### 컨볼루션 레이어

In [5]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

#### 풀링 레이어

In [6]:
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 완전 연결 레이어

2개의 CNN 레이어를 통해 추출된 7 * 7 * 64개의 특징들을 이용해서 최종 목표인 10개로 분류하는 레이어를 만든다.

In [7]:
# 7 * 7 * 64: CNN 레이어의 아웃풋 개수
# 256: 최종 출력(10개)의 바로 전 히든 레이어의 뉴런 개수
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
# 결과는 10개의 1차원 배열이므로 입력으로, 들어온 3차원 배열 값을 1차원으로 변형
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

# 고수준 API를 이용하면 더 간단하게도 가능하다
# L3 = tf.contrib.layers.flatten(L2)
# L3 = layers.dense(L3, 256, activation=tf.nn.relu)

L3 = tf.nn.dropout(L3, keep_prob)

In [8]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

# 고수준 API를 이용하면 더 간단하게도 가능하다
# model = layers.dense(L3, 10, activation=None)

모델 작성 끝!

### 학습

손실 함수와 최적화 함수를 작성한다.

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
# 나중에 이 최적화 함수로도 테스트 해보자
# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

학습을 실행하는 코드를 작성한다.

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 모델에 입력을 전달해주기 위해서 값을 변형
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={
            X: batch_xs,
            Y: batch_ys,
            keep_prob: 0.7,
        })
        
        total_cost += cost_val
    
    print('Epoch:', '%04d' % (epoch + 1), 'Avg cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy, feed_dict={
    X: mnist.test.images.reshape(-1, 28, 28, 1),
    Y: mnist.test.labels,
    keep_prob: 1,
}))

Epoch: 0001 Avg cost = 0.358
Epoch: 0002 Avg cost = 0.117
Epoch: 0003 Avg cost = 0.085
Epoch: 0004 Avg cost = 0.063
Epoch: 0005 Avg cost = 0.052
Epoch: 0006 Avg cost = 0.044
Epoch: 0007 Avg cost = 0.039
Epoch: 0008 Avg cost = 0.034
Epoch: 0009 Avg cost = 0.030
Epoch: 0010 Avg cost = 0.025
Epoch: 0011 Avg cost = 0.024
Epoch: 0012 Avg cost = 0.021
Epoch: 0013 Avg cost = 0.020
